<a href="https://colab.research.google.com/github/ThGreatExplorer/NOVA/blob/main/rnn_nustar_smote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#google colab pro **
#RNN - this time with smote (adasyn) -- hi i'm old (12/1 at 11:23pm, please look for updated one)
#packages
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from imblearn.over_sampling import ADASYN

#Data
with h5py.File('/mnt/e/BHvsPUL/small_1800.h5', 'r') as f:  #nolan's smaller databite
    phi = f['phi'][:]   #light curves
    y = f['is_pulsar'][:]

X = phi.reshape(phi.shape[0], phi.shape[1], 1)  #need 3d input for rnn

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Pulsars: {np.sum(y)}, Black holes: {np.sum(1-y)}")

adasyn = ADASYN(sampling_strategy='minority')
X_res, y_res = adasyn.fit_resample(X, y)
print(y_res.value_counts())


# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y
)
#80 % train, 20 % test splits
#stratify to not lose pulsar vs bh
# Normalize
n_samples, seq_len, n_features = X_train.shape
X_train_reshaped = X_train.reshape(-1, n_features)
X_test_reshaped = X_test.reshape(-1, n_features)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(n_samples, seq_len, n_features)
X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

# Build model
model = models.Sequential(name='NuSTAR_Classifier')
#64 units
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True),   #going back and forward - want this bc of pulsar periodicy
                               input_shape=(seq_len, n_features)))
model.add(layers.Dropout(0.3)) #turning off 30% neurons during training
#32 units
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=True)))  #passing full sequence with return_sequence
model.add(layers.Dropout(0.3))
#16 units
model.add(layers.LSTM(16))
model.add(layers.Dropout(0.4))
#32 units
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.3))
#16 units
model.add(layers.Dense(16, activation='relu'))
#single vector for output
model.add(layers.Dense(1, activation='sigmoid')) #0 = black hole, 1 = pulsar

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

print(model.summary())

# Train
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),  #patience for 50, rn doing 10 so doesnt really matter lol
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6) #if loss plateaus at 5 epochs cutting learning rate in half
]

history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

# Evaluate
test_loss, test_acc, test_auc = model.evaluate(X_test_scaled, y_test)
print(f"\nTest Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# Plot training history
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['accuracy'], label='Train')
axes[1].plot(history.history['val_accuracy'], label='Val')
axes[1].set_title('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(history.history['auc'], label='Train')
axes[2].plot(history.history['val_auc'], label='Val')
axes[2].set_title('AUC')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Confusion matrix and ROC
y_pred_proba = model.predict(X_test_scaled)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()   #greater than 0.5 = pulsar :)

cm = confusion_matrix(y_test, y_pred)
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Black Hole', 'Pulsar']))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Confusion matrix
im = axes[0].imshow(cm, cmap='Blues')
axes[0].set_xticks([0, 1])
axes[0].set_yticks([0, 1])
axes[0].set_xticklabels(['Black Hole', 'Pulsar'])
axes[0].set_yticklabels(['Black Hole', 'Pulsar'])
axes[0].set_xlabel('Predicted')
axes[0].set_ylabel('True')
axes[0].set_title('Confusion Matrix')
for i in range(2):
    for j in range(2):
        axes[0].text(j, i, cm[i, j], ha="center", va="center")
plt.colorbar(im, ax=axes[0])             #want result of >0.9

# ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc = auc(fpr, tpr)
axes[1].plot(fpr, tpr, lw=2, label=f'AUC = {roc_auc:.3f}')
axes[1].plot([0, 1], [0, 1], 'k--', lw=2)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('ROC Curve')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Save model
model.save('/mnt/e/BHvsPUL/nustar_rnn_classifier_smallV2.keras')